## ***Módulo 1 - Utilización, procesamiento y visualiazción de grandes volúmenes de datos.***

### Mariana Castro Payns - A01706038

DataSet obtenido de: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=Books_rating.csv

In [1]:
# Importar y establecer variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/mariana/spark/spark-3.2.2-bin-hadoop3.2"

In [2]:
# Buscar e inicializar instalación Spark
import findspark
findspark.init()
findspark.find()

'/home/mariana/spark/spark-3.2.2-bin-hadoop3.2'

In [3]:
# Inicializar sesión de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "5g").appName('Books').getOrCreate()

22/11/02 23:24:11 WARN Utils: Your hostname, mariana-Victus resolves to a loopback address: 127.0.1.1; using 192.168.1.74 instead (on interface wlo1)
22/11/02 23:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/02 23:24:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Obtener App Name y Master como verificación de la sesión
print("APP Name :"+spark.sparkContext.appName)
print("Master :"+spark.sparkContext.master)

APP Name :Books
Master :local[*]


In [5]:
# Leer archivo e indicar el encabezado
df = spark.read.option('header', 'true').csv('file:///home/mariana/IA/Books_rating.csv', inferSchema=True)

In [6]:
# Mostrar el data frame cargado
df.show()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| null| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| null|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| null|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

In [7]:
# Imprimir el esquema de las variables
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [8]:
# Cambiar tipos de datos en columnas necesarias
from pyspark.sql.functions import column
df = df.withColumn("label", column("review/score").cast('int'))
df = df.withColumn("Price", column("price").cast('int'))

In [9]:
# Verificar tipos de dato
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- label: integer (nullable = true)



In [10]:
# Determinar valores nulos
from pyspark.sql.functions import when,lit,count,isnan,col
df.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+-----+
| Id|Title|  Price|User_id|profileName|review/helpfulness|review/score|review/time|review/summary|review/text|label|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+-----+
|  0|  208|2519269| 562250|     562250|               367|         130|         27|            65|         43|18064|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+-----+



In [11]:
# Eliminar valores nulos
df = df.na.drop()

In [12]:
# Seleccionar columnas relevantes
df = df.select("id", "review/summary", "label")

In [13]:
# Mostrar selección
df.show()

+----------+--------------------+-----+
|        id|      review/summary|label|
+----------+--------------------+-----+
|0829814000|Outstanding Resou...|    5|
|0829814000|Small Churches CA...|    5|
|0829814000|Not Just for Past...|    5|
|0829814000|Small church past...|    5|
|0595344550|            not good|    1|
|0595344550|  Here is my opinion|    4|
|0595344550|        Buyer beware|    1|
|0595344550| Fall on your knee's|    5|
|0595344550|      Bravo Veronica|    5|
|0595344550|           Wonderful|    5|
|0595344550|           Awesome !|    5|
|0595344550|      Glorious story|    5|
|0595344550|         Loved it !!|    5|
|0595344550|Five stars it not...|    5|
|0595344550|    A FIVE STAR BOOK|    5|
|0595344550|Whispers of the W...|    5|
|0595344550|Errors, but great...|    4|
|0595344550|          The Worst!|    1|
|0595344550|The truth about W...|    5|
|0595344550|How could someone...|    5|
+----------+--------------------+-----+
only showing top 20 rows



In [14]:
# Importar librerías necesarias
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Configuración de fases del Pipeline - Tokenizer, Hashing, Regresion Lineal
tokenizer = Tokenizer(inputCol="review/summary", outputCol="words")
hashing = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [15]:
lr = LinearRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashing, lr])

In [16]:
# Importar librerías necesarias
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

# Construcción de un grid de parametros 
paramGrid = ParamGridBuilder()\
    .addGrid(hashing.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

In [17]:
# Importar librerías necesarias
from pyspark.ml.evaluation import RegressionEvaluator

# Configurar el validador Train-Validation
tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 75% de los datos sera utilizado para el training.
                           trainRatio=0.75)

In [18]:
# Configurar el cross validator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

In [19]:
# Dividir el data ser
train, test = df.randomSplit([0.75,0.25])

In [20]:
# Se ejecuta la validación cruzada para encontrar los mejores parámetros de configuración.
cvModel = crossval.fit(train)

22/11/02 23:24:47 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/11/02 23:25:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/02 23:25:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/11/02 23:25:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/02 23:25:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [21]:
# Obtener predicciones con validación cruzada de 20 datos (dado el tamaño del dataset)
predictions = cvModel.transform(test)
sel = predictions.select("id", "label", "prediction")
for row in sel.take(20):
    print(row)

Row(id='002412141X', label=3, prediction=4.2465028949999954)
Row(id='0027613909', label=5, prediction=4.592399859148961)
Row(id='0027613909', label=4, prediction=4.333770252312783)
Row(id='0028642627', label=4, prediction=4.197479836883434)
Row(id='0028642627', label=1, prediction=3.130675096513806)
Row(id='0028642627', label=4, prediction=4.390831234535854)
Row(id='002864266X', label=4, prediction=4.15496411229088)
Row(id='002864266X', label=5, prediction=4.232667706795603)
Row(id='002864266X', label=5, prediction=4.851200454923863)
Row(id='002864266X', label=5, prediction=4.418851226719356)
Row(id='002864266X', label=3, prediction=3.7111867706606865)
Row(id='002864266X', label=4, prediction=3.5630599533900984)
Row(id='002864266X', label=2, prediction=3.871474744391057)
Row(id='002864266X', label=5, prediction=3.554910418071861)
Row(id='0028642686', label=5, prediction=4.891285810891735)
Row(id='0030651786', label=5, prediction=4.045040193774086)
Row(id='0060005416', label=5, predicti

In [22]:
# Buscar la mejor configuración con el Train-Validation
tvsModel = tvs.fit(train)

In [23]:
# Realizar predicciones con los datos de test
tvsModel.transform(test).select("id", "label", "prediction").show(20)

+----------+-----+------------------+
|        id|label|        prediction|
+----------+-----+------------------+
|002412141X|    3|4.2465028949999954|
|0027613909|    5| 4.592399859148961|
|0027613909|    4| 4.333770252312783|
|0028642627|    4| 4.197479836883434|
|0028642627|    1| 3.130675096513806|
|0028642627|    4| 4.390831234535854|
|002864266X|    4|  4.15496411229088|
|002864266X|    5| 4.232667706795603|
|002864266X|    5| 4.851200454923863|
|002864266X|    5| 4.418851226719356|
|002864266X|    3|3.7111867706606865|
|002864266X|    4|3.5630599533900984|
|002864266X|    2| 3.871474744391057|
|002864266X|    5| 3.554910418071861|
|0028642686|    5| 4.891285810891735|
|0030651786|    5| 4.045040193774086|
|0060005416|    5| 5.129980865648127|
|0060226064|    5| 4.348112876804524|
|0060226064|    3| 3.512606115014804|
|0060226064|    5| 4.512661706897008|
+----------+-----+------------------+
only showing top 20 rows

